# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from datasets import load_dataset

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("imdb")

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token

tokenized_dataset = {}
for split in ["train", "test"]:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True), batched=True
    )

config.json: 100%|██████████| 718/718 [00:00<00:00, 1.26MB/s]
vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 15.5MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 39.1MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 45.2MB/s]
Map: 100%|██████████| 25000/25000 [00:26<00:00, 933.75 examples/s]


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2-medium",
    num_labels=2,
    id2label={0: "neg", 1: "pos"},
    label2id={"neg": 0, "pos": 1}
)

for param in model.parameters():
    param.requires_grad = True

model.safetensors:  12%|█▏        | 189M/1.52G [00:00<00:06, 208MB/s] 

In [ ]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/neg_pos",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
# from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

In [ ]:
# peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, target_modules=['c_attn', 'c_proj'])

# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

In [ ]:
tokenized_dataset = {}
for split in ["train", "test"]:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["content"], padding=True, truncation=True), batched=True
    )
    
tokenized_dataset["train"].set_format("torch", columns=["label", "input_ids", "attention_mask"])
tokenized_dataset["test"].set_format("torch", columns=["label", "input_ids", "attention_mask"])

In [ ]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [ ]:
# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/movie_sentiment",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

# trainer.train()

In [ ]:
trainer.train()

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.